<a href="https://colab.research.google.com/github/prawees/marmoset_project/blob/main/MarmosetProject_6402001_06_09_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip -O marmoset_data.zip
#Download dataset and name it marmoset_data.zip
!unzip marmoset_data.zip -d ./marmoset
#Unzip file
!pip install ultralytics deeplabcut2yolo
#Install ultralytics and deeplabcut2yolo

In [ ]:
import deeplabcut2yolo

json = "./marmoset/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
#path to the DeepLabCut annotation file (.json format)
csv = "./marmoset/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
#path to the .csv file
save = "./marmoset/marmoset-dlc-2021-05-07/labeled-data/"
#the directory where the converted YOLO-compatible data will be saved

deeplabcut2yolo.convert(json, csv, save, datapoint_classes=[0, 1], n_keypoint_per_datapoint=30, precision=6)
#Use fucntion deeplabcut2yolo.convert() to convert deeplabcut data into YOLO-compatible data
#datapoint_classes = classes labels for the datapoints (in this case there are two monkeys to detect)
#n_keypoint_per_datapoint = number of keypoints per datapoint
#precision = decimal precision for the converted annotations

In [ ]:
import yaml

data = {
    "train": "/content/marmoset/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1",
    #path to the data that will use to train the model
    "val": "/content/marmoset/marmoset-dlc-2021-05-07/labeled-data/refinement1",
    #path to the data that will use to validate the model
    "kpt_shape": [15, 3],
    #keypoint shape: in this case there are 15 keypoints per class  and each keypoints hold 3 values (x, y, confidence score)
    "flip_idx":  [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14],
    #flip index: defines which keypoints should be swapped when the image is horizontally flipped (mirrored)
    "nc": 2,
    #number of classes
    "names" : ["B", "W"]
    #name of classes
    }

output= "/content/output.yaml"
#the directory where the yml data will be saved

with open(output, "w") as file:
    yaml.dump(data, file, default_flow_style=None, sort_keys=False)

In [ ]:
from ultralytics import YOLO

modelyolo = YOLO("yolov8n-pose.pt")

modelyolo.train(data="output.yaml", epochs=20)
#train YOLO model

In [ ]:
!wget https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip -O marmoset_test.zip
!unzip marmoset_test.zip -d ./marmoset_test

In [ ]:
from ultralytics import SAM

# Load a model
modelsam = SAM("sam_b.pt")

In [ ]:
import os

# Define input and output directories
input_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
output_dir = "./marmoset_test/marmoset-dlc-2021-05-07/labeled-data/output"

# Create output directory if it doesn't exist
if os.path.exists(output_dir) == False:
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png')):
        # Path to the image
        image_path = os.path.join(input_dir, filename)

        # Run YOLO model
        model_output = modelyolo(image_path)
        boxes = model_output[0].boxes.xyxy

        # Check bounding boxes
        if len(boxes) == 0:
            print(f"No {filename}")
            continue

        # Convert bounding boxes to list format suitable for SAM
        bboxes_list = boxes[0].cpu().numpy().tolist()  # Convert to numpy array and then to list

        # Run SAM model with bounding boxes
        resultsam = modelsam(image_path, bboxes=bboxes_list)

        # Save the annotated image
        result = resultsam[0]
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_a.png")
        result.save(output_path)
        print(f"{output_path} saved. \n")